In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
import string
import time
from collections import Counter
import nltk
from time import sleep
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
headers = {'User-Agent': ua.random}

nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Гриша\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_reviews(url): # функция для парсинга даты с кинопоиска
    time.sleep(5)
    reviews = []
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    time.sleep(5)
    html = session.get(url, headers=headers)
    page = html.text
    soup = BeautifulSoup(page, 'html.parser')
    for i in soup.find_all('div', {'class':'brand_words'}):
        reviews.append(i.text)
    return reviews

In [3]:
def preproc(rev): # предобработка данных
    lemm_words = []
    rev = rev.lower()
    for i in '\n\r\t«»-…':
        rev = rev.replace(i, '')
    for i in string.punctuation:
        rev = rev.replace(i, '')
    words = [morph.parse(x)[0].normal_form for x 
             in nltk.word_tokenize(rev)] 
    for i in words:
        lemm_words.append(i)
    return lemm_words

In [4]:
def divide_data(good_reviews_raw, bad_reviews_raw): # разделение даты на трейнинг и тест
    good_train = []
    bad_train = []
    test_data = {}
    upd = {}
    for i in good_reviews_raw[:60]:
        for j in preproc(i):
            good_train.append(j)
    for i in bad_reviews_raw[:60]:
        for j in preproc(i):
            bad_train.append(j)
    for i in good_reviews_raw[61:71]:
        test_data[i] = 'positive'
    for i in bad_reviews_raw[61:71]:
        upd[i] = 'negative'
    test_data.update(upd)
    return good_train, bad_train, test_data

In [5]:
def get_word_set(reviews): # убираем стоп-слова и шум и делаем множество слов
    stop = []
    noise = []
    russian_stopwords = stopwords.words("russian")
    for i in reviews:
        if i not in russian_stopwords:
            stop.append(i)
    for i in Counter(stop).items():
        if i[1] >= 2:
            noise.append(i[0]) 
    return noise

In [6]:
def predict_review(review, good_words, bad_words): # предсказание
    score = {}
    score['positive'] = 0
    score['negative'] = 0
    for w in review:
        if w in good_words:
            score['positive'] += 1
        elif w in bad_words:
            score['negative'] += 1
    return Counter(score).most_common()

In [7]:
def only(good_words, bad_words): # делаем множества для хороших и плохих слов
    good_only = []
    bad_only = []
    for i in good_words:
        if i not in bad_words:
            good_only.append(i)
    for i in bad_words:
        if i not in good_words:
            bad_only.append(i)
    return good_only, bad_only

In [8]:
def test_model(test_data, good_only, bad_only): # тест модели
    gold = []
    results = []
    for i in test_data.items():
        results.append(predict_review(preproc(i[0]), good_only, bad_only)[0][0])
        gold.append(i[1])
    print("RESULTS:")
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [9]:
good_url = '''https://www.kinopoisk.ru/film/447301/reviews/ord/date/status/good/perpage/100/'''
bad_url = '''https://www.kinopoisk.ru/film/447301/reviews/ord/date/status/bad/perpage/100/'''
good_train, bad_train, test_data = divide_data(
    get_reviews(good_url), get_reviews(bad_url))
good_only, bad_only = only(
    get_word_set(good_train), get_word_set(bad_train))


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/">
<head profile="http://gmpg.org/xfn/11">
<title>Начало — отзывы и рецензии — КиноПоиск</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="121953784483000" property="fb:app_id"/>
<meta content="#FF6600" name="msapplication-TileColor"/>
<meta content="https://st8.kp.yandex.net/public/img/favicons/favicon-144.png" name="msapplication-TileImage"/>
<meta content="Кинопоиск" name="application-name"/>
<meta content="https://st2.kp.yandex.net/public/xml/ieconfig.xml" name="msapplication-config"/>
<meta content="width=960" name="viewport">
<meta content="1


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" lang="ru" prefix="og: http://ogp.me/ns# video: http://ogp.me/ns/video# ya: http://webmaster.yandex.ru/vocabularies/" xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/">
<head profile="http://gmpg.org/xfn/11">
<title>Начало — отзывы и рецензии — КиноПоиск</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="121953784483000" property="fb:app_id"/>
<meta content="#FF6600" name="msapplication-TileColor"/>
<meta content="https://st8.kp.yandex.net/public/img/favicons/favicon-144.png" name="msapplication-TileImage"/>
<meta content="Кинопоиск" name="application-name"/>
<meta content="https://st2.kp.yandex.net/public/xml/ieconfig.xml" name="msapplication-config"/>
<meta content="width=960" name="viewport">
<meta content="1

In [11]:
test_model(test_data, good_only, bad_only)

positive positive
positive positive
positive positive
positive positive
positive positive
positive positive
positive positive
positive positive
positive positive
positive positive
negative positive
negative positive
negative negative
negative positive
negative negative
negative negative
negative positive
negative negative
negative negative
negative negative
RESULTS:
Accuracy: 0.8000


# Улучшения
### 1. Классификатор
Нужно будет обязательно сделать классификатор основанный на методах машинного обучения и векторизации слов, для лучшего распознавания также может помочь анализ не отдельных слов, а словосочетаний
### 2. Больше данных
Для лучшего распознавания (тем более для нейросети) нужно больше данных для обучения
### 3. Обход капчи
У кинопоиска капча и я не нашел API, можно найти способ обойти (ну это уже больше к парсингу)